In [53]:
from owlready2 import owlready2, get_ontology, sync_reasoner_pellet, Thing, Not, And, default_world, AllDifferent, AllDisjoint, Imp

In [54]:
owlready2.JAVA_EXE='/usr/bin/java'

In [55]:
TMP_FOLDER = './tmp/'

In [56]:
onto = get_ontology('http://students.org/') # new ontology, providing its IRI
with onto:
    # Define a Person
    class Person(Thing):  # new subclass of Thing
        pass
    
    # Define a Course
    class Course(Thing):
        # subclass of not a Person
        is_a = [Not(onto.Person)]
        
    # AllDisjoint([Person, Course])
    
    # A Person can teach a Course
    class teaches(Person >> Course): pass
    
    # Define a Teacher
    class Teacher(Thing):
        # as a Person who teaches some Course
        equivalent_to = [And([Person, teaches.some(Course)])]
        
    # A Person can attend a Course    
    class attends(Person >> Course): pass
    
    # Define a Student
    class Student(Thing):
        # as a Person who attends some Course
        equivalent_to = [And([Person, attends.some(Course)])]

In [57]:
with onto:
    print(Person.ancestors()) # Print every superclasses

{owl.Thing, students.org.Person}


In [58]:
# Define some individuals
with onto:
    mary = Person(name='mary') # mary is instance of class Personne
    cs600 = Course(name='cs600')
    alice = Person(name='alice')
    alice.teaches = [cs600]
    mary.attends = [cs600]

In [59]:
 # Check what are alice and mary
with onto:
    print(alice, alice.is_a)
    print(mary, mary.is_a)

students.org.alice [students.org.Teacher, students.org.Woman, students.org.Woman]
students.org.mary [students.org.Student, students.org.Woman, students.org.Woman]


In [60]:
with onto:
    for prop in mary.get_properties():   # Each properties of Mary with their values
        for value in prop[onto.mary]:
            print(".%s == %s" % (prop.name, value))


.attends == students.org.cs600


In [61]:
onto.save(TMP_FOLDER + "students.owl") # This save is bug-prone - generation of <stud:Person rdf:about="alice">

In [62]:
default_world.save(TMP_FOLDER + 'world.owl') # generation of <stud:Person rdf:about="http://students.org/alice">

In [63]:
# Synchronizing reasoner
sync_reasoner_pellet([onto], infer_property_values=True, infer_data_property_values=True, debug=True, keep_tmp_file=True)

* Owlready2 * Running Pellet...
    /usr/bin/java -Xmx2000M -cp /home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/commons-codec-1.6.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/slf4j-log4j12-1.6.4.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/jena-iri-0.9.5.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/jena-arq-2.10.0.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/xml-apis-1.4.01.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/antlr-3.2.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/jcl-over-slf4j-1.6.4.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/jena-core-2.10.0.jar:/home/william/Progra

In [64]:
with onto:
    # Check what are alice and mary after Pellet
    print(alice, alice.is_a)
    print(mary, mary.is_a)


students.org.alice [students.org.Teacher, students.org.Woman, students.org.Woman]
students.org.mary [students.org.Student, students.org.Woman, students.org.Woman]


In [65]:
with onto:
    # Define a new Course
    cs101 = Course(name='cs101')
    
    # Define bob as a Person
    bob = Person(name='bob')
    # who is teaching cs101
    bob.teaches = [cs101]
    # and attending cs600
    bob.attends = [cs600]
    
# print what is bob
print(bob.is_a)

[students.org.Teacher, students.org.Student, students.org.Man, students.org.Man]


In [66]:
# Synchronizing reasoner
sync_reasoner_pellet([onto], infer_property_values=True, infer_data_property_values=True, debug=True, keep_tmp_file=True)

* Owlready2 * Running Pellet...
    /usr/bin/java -Xmx2000M -cp /home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/commons-codec-1.6.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/slf4j-log4j12-1.6.4.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/jena-iri-0.9.5.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/jena-arq-2.10.0.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/xml-apis-1.4.01.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/antlr-3.2.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/jcl-over-slf4j-1.6.4.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/jena-core-2.10.0.jar:/home/william/Progra

In [67]:
# Check what is bob
bob.is_a

[students.org.Teacher,
 students.org.Student,
 students.org.Man,
 students.org.Man]

In [68]:
# Check if any inconsistent classes
print(list(onto.world.inconsistent_classes()))

[]


In [69]:
# Add some new relation
with onto:
    class Man(Thing):
        is_a = [Person]
    class Woman(Thing):
        is_a = [Person]
        
    # Man and Woman are disjoint (no transgender)
    AllDisjoint([Man,Woman])        
    
    class has_sibling(Person >> Person): pass
    
    bob.is_a.append(Man)
    alice.is_a.append(Woman)
    mary.is_a.append(Woman)

In [70]:
# Synchronizing reasoner
sync_reasoner_pellet([onto], infer_property_values=True, infer_data_property_values=True, debug=True, keep_tmp_file=True)

* Owlready2 * Running Pellet...
    /usr/bin/java -Xmx2000M -cp /home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/commons-codec-1.6.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/slf4j-log4j12-1.6.4.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/jena-iri-0.9.5.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/jena-arq-2.10.0.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/xml-apis-1.4.01.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/antlr-3.2.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/jcl-over-slf4j-1.6.4.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/jena-core-2.10.0.jar:/home/william/Progra

In [71]:
bob.is_a

[students.org.Teacher,
 students.org.Student,
 students.org.Man,
 students.org.Man,
 students.org.Man]

In [ ]:
# Creating the Imply rules
with onto:
    imp = Imp()

In [72]:
# Defining a new rule (transitivity) : Two different Person having a common sibling different from them are siblings themselves
with onto:
    imp.set_as_rule("Person(?x), Person(?y), Person(?z), \
                     DifferentFrom(?x, ?y), DifferentFrom(?y, ?z), DifferentFrom(?x, ?z), \
                     has_sibling(?x, ?y), has_sibling(?y, ?z) -> has_sibling(?x, ?z)")

In [73]:
# Ensure that alice, bob, mary are not the same
AllDifferent([bob, alice, mary])

AllDisjoint([students.org.bob, students.org.alice, students.org.mary])

In [74]:
# Add some siblings
with onto:
    bob.has_sibling = [alice]
    alice.has_sibling = [mary]

In [75]:
# Synchronizing reasoner
sync_reasoner_pellet([onto], infer_property_values=True, infer_data_property_values=True, debug=True, keep_tmp_file=True)

* Owlready2 * Running Pellet...
    /usr/bin/java -Xmx2000M -cp /home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/commons-codec-1.6.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/slf4j-log4j12-1.6.4.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/jena-iri-0.9.5.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/jena-arq-2.10.0.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/xml-apis-1.4.01.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/antlr-3.2.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/jcl-over-slf4j-1.6.4.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/jena-core-2.10.0.jar:/home/william/Progra

* Owlready * Adding relation students.org.bob has_sibling students.org.esther


* Owlready2 * Pellet took 3.3277955055236816 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [76]:
# The reasoner should have deduced that bob has mary as sibling
bob.has_sibling

[students.org.mary, students.org.alice, students.org.esther]

In [77]:
# Creating a new relation : has_sister
with onto:
    class has_sister(has_sibling):
        domain = [Person]
        range = [Woman]

In [78]:
# Checking has_sister inherits correctly from has_sibling
with onto:
    esther = Woman(name = 'esther')
    bob.has_sister = [esther]
    print(bob.has_sister)
    print(bob.has_sibling)

[students.org.esther]
[students.org.mary, students.org.alice, students.org.esther]


In [79]:
# Defining a new rule (reciprocity) : Two different Person having a common sibling different from them are siblings themselves
with onto:
    imp.set_as_rule("Person(?x), Person(?y),\
                     DifferentFrom(?x, ?y),\
                     has_sibling(?x, ?y) -> has_sibling(?y, ?x)")

In [81]:
# Synchronizing reasoner
sync_reasoner_pellet([onto], infer_property_values=True, infer_data_property_values=True, debug=True, keep_tmp_file=True)

* Owlready2 * Running Pellet...
    /usr/bin/java -Xmx2000M -cp /home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/commons-codec-1.6.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/slf4j-log4j12-1.6.4.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/jena-iri-0.9.5.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/jena-arq-2.10.0.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/xml-apis-1.4.01.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/antlr-3.2.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/jcl-over-slf4j-1.6.4.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/jena-core-2.10.0.jar:/home/william/Progra

* Owlready * Adding relation students.org.mary has_sibling students.org.bob
* Owlready * Adding relation students.org.mary has_sibling students.org.alice
* Owlready * Adding relation students.org.esther has_sibling students.org.bob
* Owlready * Adding relation students.org.alice has_sibling students.org.bob


* Owlready2 * Pellet took 3.8001279830932617 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [83]:
# Checking latter rule : alice should have bob as sibling
with onto:
    print(alice.has_sibling)

[students.org.mary, students.org.bob]


In [85]:
# Defining a new rule : A person having a Woman sibling has a sister
with onto:
    imp.set_as_rule("Person(?x), Person(?y),\
                     has_sibling(?x, ?y), Woman(?y) -> has_sister(?x, ?y)")

In [86]:
# Synchronizing reasoner
sync_reasoner_pellet([onto], infer_property_values=True, infer_data_property_values=True, debug=True, keep_tmp_file=True)

* Owlready2 * Running Pellet...
    /usr/bin/java -Xmx2000M -cp /home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/commons-codec-1.6.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/slf4j-log4j12-1.6.4.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/jena-iri-0.9.5.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/jena-arq-2.10.0.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/xml-apis-1.4.01.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/antlr-3.2.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/jcl-over-slf4j-1.6.4.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/home/william/Programming/OWL/.venv/lib/python3.8/site-packages/owlready2/pellet/jena-core-2.10.0.jar:/home/william/Progra

* Owlready * Adding relation students.org.bob has_sister students.org.mary
* Owlready * Adding relation students.org.bob has_sister students.org.alice
* Owlready * Adding relation students.org.alice has_sister students.org.mary


* Owlready2 * Pellet took 3.264251232147217 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [87]:
# Checking latter rule :  bob should have alice, mary and esther as sisters
with onto:
    print(bob.has_sister)

[students.org.esther, students.org.mary, students.org.alice]


In [39]:
with onto:
    Thing.descendants(include_self = True, only_loaded = False)

In [40]:
onto.save(TMP_FOLDER + 'students.owl')

In [41]:
default_world.ontologies

{'http://anonymous/': get_ontology("http://anonymous/"),
 'http://students.org/': get_ontology("http://students.org/"),
 'http://inferrences/': get_ontology("http://inferrences/")}

In [42]:
default_world.save(TMP_FOLDER + 'world.owl')

Native SPARQL engine (see [SPARQL Owlready2 documentation](https://owlready2.readthedocs.io/en/latest/sparql.html#native-sparql-engine))

RDFLib SPARQL engine

In [88]:
with onto:
    print(bob.iri)

http://students.org/bob


In [90]:
graph = default_world.as_rdflib_graph()
graph.bind('owl', 'http://www.w3.org/2002/07/owl#')

In [91]:
r = list(graph.query('SELECT ?classe WHERE { <http://students.org/bob> a ?classe . ?classe a owl:Class . }'))
print(r)

[(rdflib.term.URIRef('http://students.org/Person'),), (rdflib.term.URIRef('http://students.org/Teacher'),), (rdflib.term.URIRef('http://students.org/Student'),), (rdflib.term.URIRef('http://students.org/Man'),)]


In [92]:
graph.bind("students", "http://students.org/")

In [93]:
r = list(graph.query('SELECT ?classe WHERE { student:bob a ?classe . ?classe a owl:Class . }'))
print(r)

[(rdflib.term.URIRef('http://students.org/Person'),), (rdflib.term.URIRef('http://students.org/Teacher'),), (rdflib.term.URIRef('http://students.org/Student'),), (rdflib.term.URIRef('http://students.org/Man'),)]


In [94]:
onto.destroy()

In [95]:
bob.has_sibling

[students.org.mary, students.org.alice, students.org.esther]

In [96]:
bob.iri

'http://students.org/bob'

In [98]:
onto.save(TMP_FOLDER + 'students.owl')

TypeError: 'NoneType' object is not subscriptable